<a href="https://colab.research.google.com/github/AngieB1988/Bigdata/blob/main/layer1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


--2022-09-18 20:00:09--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.99MB/s    in 0.2s    

2022-09-18 20:00:10 (4.99 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in data from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), inferSchema=True, sep='\t')
df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...|2015-08-31 00:00:00|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...| 

In [ ]:
print(df.count())
df =df.dropna();
print(df.count())

9002021
9001052


In [ ]:
print(df.count())
df=df.drop_duplicates()
print(df.count())

9001052
9001052


In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType
df =df.withColumn('review_date',col('review_date').cast(DateType()))

In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
review_id_df= df.select(['review_id','customer_id','product_id','product_parent','review_date'])
review_id_df.show

<bound method DataFrame.show of DataFrame[review_id: string, customer_id: int, product_id: string, product_parent: int, review_date: date]>

In [ ]:
products_df = df.select(['product_id','product_title'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B008KYY0ZE|Garmin Approach S...|
|B002LITI70|Magellan RoadMate...|
|B00P07H97A|Motorola Droid Tu...|
|B004I58HJO|Insten Black Leat...|
|B00ZWZ1G7E|iDeaUSA Bluetooth...|
|B0015RB39O|Delton Platinum U...|
|B00EMY297M|DESIGNCOOL a spec...|
|B005C5ZF1W|Nokia 3711 Brown ...|
|B00GHDLR4C|[NFC/Google Walle...|
|B00BHI40OQ|iCracked Premium ...|
|B004E10KI8|      Jawbone JAMBOX|
|B00NXEX8D6|Towallmark(TM) St...|
|B0055Q5F2W|Otterbox 08-2089 ...|
|B00I8RVKZ6|iPhone 6 Case, Ma...|
|B007K0G3S6|Samsung Galaxy S ...|
|B00HTT4UOM|Sannysis(TM) 330P...|
|B00EPMOIKC|LG G2 Case, Diy L...|
|B00ATNSFD2|E LV Deluxe PU Le...|
|B00SI9WXYG|Samsung Gusto 2, ...|
|B002QIOUR8|New Demon Silicon...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
print(products_df.count())
products_df=products_df.drop_duplicates()
print(products_df.count())

9001052
906482


In [ ]:
df.select(['customer_id']).distinct().count()

5197356

In [ ]:
customers_df=df.groupby('customer_id').agg(count('customer_id').alias('customer_count'))

In [ ]:
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   41519338|             1|
|   32618195|             2|
|    1440641|             5|
|   31315080|             1|
|   25855821|             1|
|    2169806|             1|
|   45035543|             1|
|   36929016|             3|
|   29689090|             3|
|   47093832|             1|
|    3897890|             2|
|    4578838|             6|
|   15320172|             2|
|   10299032|             1|
|   40985731|            13|
|   33312603|             2|
|   12164957|             5|
|   19094231|             1|
|    1254726|             1|
|   12492302|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1002TBBY7I096|          5|            0|          0|   N|
|R1007HHGRAOJI3|          5|            1|          1|   N|
|R100B1VOG8IFX7|          5|            1|          1|   N|
|R100COFLOW3VGC|          3|            0|          0|   N|
|R100D3V7QQPGRH|          5|            0|          0|   N|
|R100FIFWZ788EN|          5|            0|          0|   N|
|R100M9LRMSIVE5|          2|            0|          0|   N|
|R100NQEWXUQZ8L|          5|            0|          0|   N|
|R100S4LYKKAAMQ|          5|            0|          0|   N|
|R10132XVXBZAV8|          5|            0|          0|   N|
|R101379P67T2N4|          3|            0|          0|   N|
|R10167ZP23S0BL|          5|            0|          0|   N|
|R1017D3K9S4VKM|          5|            0|          0|   N|
|R101BBTKD9CGLE|          5|            

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://myprostgredb.cdhbjqbvqosf.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}